# 🗑️ ЧЕСТНАЯ VLM для распознавания мусора

**Архитектура:** ВСЯ CV часть = ваш ансамбль (YOLO + RT-DETR)

```
Изображение → Ваш Ансамбль → Детекции → Ответ
```

⚠️ **Важно:** Ансамбль обучен только на классы мусора (glass, plastic, metal, paper, organic).  
Для описания сцены (трава, дорога) нужно дообучить ансамбль на классы окружения!


In [ ]:
# 1. Установка
!git lfs install
!git clone https://github.com/sry4theweight/VLM_garbage.git
%cd VLM_garbage
!pip install -q ultralytics supervision transformers pillow roboflow

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


In [ ]:
# 2. Скачать датасет
!python download_roboflow_dataset.py


In [ ]:
# 3. Загрузка ЧЕСТНОЙ VLM (только ваш ансамбль!)
from VLM_Honest import HonestVLM

vlm = HonestVLM(
    yolo_path="models/yolo/yolov8x/best.pt",
    detr_path="models/rt-detr/rt-detr-101/m",
    detr_processor_path="models/rt-detr/rt-detr-101/p",
    conf_threshold=0.5
)


In [ ]:
# 4. Собираем изображения БЕЗ аугментаций
from pathlib import Path
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

valid = Path("data/roboflow_dataset/valid")
test = Path("data/roboflow_dataset/test")
images = list(valid.glob("*.jpg")) + list(test.glob("*.jpg"))
print(f"✅ {len(images)} изображений (valid+test, без аугментаций)")


In [ ]:
# 5. Функция тестирования
COLORS = {'glass':'green', 'plastic':'red', 'metal':'cyan', 'paper':'yellow', 'organic':'lime'}

def test_vlm(img_path):
    img = Image.open(img_path).convert('RGB')
    dets = vlm.detect(img)
    
    fig, ax = plt.subplots(figsize=(10,7))
    ax.imshow(img)
    for d in dets:
        b = d['box']
        c = COLORS.get(d['label'], 'white')
        ax.add_patch(patches.Rectangle((b[0],b[1]), b[2]-b[0], b[3]-b[1], lw=2, ec=c, fc='none'))
        ax.text(b[0], b[1]-5, f"{d['label']} {d['confidence']:.0%}", color=c, fontsize=9, weight='bold')
    ax.axis('off')
    plt.show()
    
    print(f"\n📝 {vlm.describe(str(img_path))}")
    for q in ["What is here?", "Is there plastic?", "How many glass?"]:
        print(f"\n❓ {q}\n💬 {vlm.answer(str(img_path), q)}")


In [ ]:
# 6. Тест на 3 случайных изображениях
for img in random.sample(images, min(3, len(images))):
    print(f"\n{'='*60}\n📷 {img.name}\n{'='*60}")
    test_vlm(img)


In [ ]:
# 7. Интерактивный тест
img = random.choice(images)
question = "Is there any metal?"  # ← Ваш вопрос!

test_vlm(img)
print(f"\n❓ {question}\n💬 {vlm.answer(str(img), question)}")


## 📌 Хотите описание сцены ("мусор на траве")?

Нужно дообучить ансамбль на классы окружения:
- `grass`, `road`, `sand`, `water`, `floor`, `table`

**Шаги:**
1. Собрать датасет с аннотациями окружения
2. Дообучить YOLO/RT-DETR на новые классы
3. Добавить классы в VLM

Тогда VLM сможет сказать: "There is plastic on the grass."


In [ ]:
# 8. Сохранение на Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/VLM_garbage
!cp -r models vlm_annotation VLM_Honest.py /content/drive/MyDrive/VLM_garbage/
print("✅ Сохранено!")
